# Sampling Images

In [4]:
import ee
import geemap

ee.Initialize()

## Soil sample data

In [ ]:
wise3 = ee.FeatureCollection("users/bingosaucer/orgc_stock_ISRIC_WISE_clip_gee")

In [57]:
Map = geemap.Map(center=(45, 0), zoom=2)

In [58]:
Map.addLayer(wise3, {}, "WISE3")

In [59]:
Map

Map(center=[45, 0], controls=(WidgetControl(options=['position'], widget=HBox(children=(ToggleButton(value=Fal…

## Merge images in collection to an image

In [24]:
def collection2image_stackbands(collection):
    '''
    Merge images in an image collection to a single image.  Each
    image in the collection becomes a band in the output image.
    
    Args:
        collection (ee.ImageCollection): Image collection.
    '''
    img = collection.iterate(lambda curr, prev: ee.Image(prev).addBands(curr), ee.Image(1))
    img = ee.Image(img)
    ibands = ee.List.sequence(1, img.bandNames().size().subtract(1))
    return img.select(ibands)


In [23]:
landcovers = ee.ImageCollection('users/bingosaucer/ConsensusLandCover_Human_Development_Percentage')

In [25]:
landcovers_img = collection2image_stackbands(landcovers)
landcovers_img.bandNames().getInfo()

['b1',
 'b1_1',
 'b1_2',
 'b1_3',
 'b1_4',
 'b1_5',
 'b1_6',
 'b1_7',
 'b1_8',
 'b1_9',
 'b1_10',
 'b1_11']

## Sampling image at points

> images + points $\rightarrow$ images' values at points

Use `ee.Image.reduceRegions` to sample the above image at a selection of points.  The result is a `ee.FeatureCollection`, where each feature is one of the selected points.  Each point has a `properties` dict, and each property inside this is the value a band of an image from the original image collection.

In [40]:
result = landcovers_img.reduceRegions(wise3, ee.Reducer.mean())

In [60]:
result.getInfo()['features'][0]

{'type': 'Feature',
 'geometry': {'type': 'Point',
  'coordinates': [-155.1005552139237, 19.744168172085583]},
 'id': '0000000000000000011a',
 'properties': {'b1': 0,
  'b1_1': 0,
  'b1_10': 0,
  'b1_11': 0,
  'b1_2': 0,
  'b1_3': 0,
  'b1_4': 23,
  'b1_5': 12,
  'b1_6': 10,
  'b1_7': 0,
  'b1_8': 5,
  'b1_9': 19,
  'bot_cm': 100,
  'datetime': 0,
  'mineral_fraction': 0.23000000417232513,
  'organic_fraction': 0.7699999809265137,
  'orgc_stock_mineral': 11.028960668150008,
  'orgc_stock_organic': 7.822259886666,
  'profile_id': 'US0004',
  'top_cm': 0}}

# Reference

- https://mygeoblog.com/2019/07/17/sample-an-image-collection/

  